# XMAGIX

- [1. Loading Library, Configuration Files and Other Stuff](#1.-Loading-Library,-Configuration-Files-and-Other-Stuff)
- [2. Starting System and Initializing Parameters](#2.-Starting-System-and-Initializing-Parameters)
- [3. Setting and Getting Parameters](#3.-Setting-and-Getting-Parameters)
  - [3.1. Get all currently set acquisition values.](#3.1.-Get-all-currently-set-acquisition-values.)
  - [3.2. Get available peaking times](#3.2.-Get-available-peaking-times)
  - [3.3. Setting Peaking Time, Bin Width and MCA-Channels](#3.3.-Setting-Peaking-Time,-Bin-Width-and-MCA-Channels)
  - [3.4. Setting Thresholds](#3.4.-Setting-Thresholds)
  - [3.5. Setting Gain](#3.5.-Setting-Gain)
- [4. Starting and Stopping a Run](#4.-Starting-and-Stopping-a-Run)
- [5. Pull Data and Plot](#5.-Pull-Data-and-Plot)

# 1. Loading Library, Configuration Files and Other Stuff
[&#x2B06;](#XMAGIX)

In [ ]:
import os
from ctypes import *
import ipywidgets as widgets
from IPython.display import display
from IPython.core.display import SVG
from rich.jupyter import print
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = "svg" # crisper plots
import numpy as np
import pandas as pd
import lmfit as lm

from xmagix import XMagix
from thestepper import *

cwd = os.getcwd()
xmagix = XMagix("lib/libhandel.so")
inifile = cwd + "/microdxp_usb2.ini"
configfile = cwd + "/config.ini"

In [ ]:
!~/klippy-env/bin/python ~/klipper/klippy/klippy.py ~/printer_data/config/printer.cfg -a /tmp/klippy_uds -l /tmp/klippy.log

In [ ]:
mdpins = [21, 20, 16]
mdpins.reverse()
detector_stepper = MyStepper(13, 19, mdpins, driver="A4988", enpin=26, steps_per_rev=200, gear_ratio=6.25, ustep=0)

In [ ]:
detector_stepper.setMotorEnable(enable=True)

In [ ]:
detector_stepper.setMotorEnable(enable=False)

In [ ]:
detector_stepper.setMicroStepping(4)

In [ ]:
detector_stepper.cwMotion(1666, 20)
detector_stepper.ccMotion(1666, 20)

In [ ]:
detector_stepper.cwMotionAngle(30, 20)
detector_stepper.ccMotionAngle(30, 20)

# 2. Starting System and Initializing Parameters
[&#x2B06;](#XMAGIX)

Available Log Levels:
```
1:  MD_ERROR
2:  MD_WARNING
3:  MD_INFO
4:  MD_DEBUG
```

In [ ]:
xmagix.setLogging(4)
xmagix.exit()
xmagix.init(inifile)
xmagix.startSystem()

# 3. Setting and Getting Parameters
[&#x2B06;](#XMAGIX)

## 3.1. Get all currently set acquisition values.
[&#x2B06;](#XMAGIX)

In [ ]:
values = xmagix.getAcquisitionValues(name="all")
values = pd.DataFrame(values.items(), columns=["Parameter", "Value"])
print(values.sort_values(by=["Parameter"]))

## 3.2. Get available peaking times
[&#x2B06;](#XMAGIX)

In [ ]:
ccurrentPeakingTimes = xmagix.getNumberOfPeakingTimes()
currentPeakingTimes = np.ctypeslib.as_array(ccurrentPeakingTimes)

## 3.3. Setting Peaking Time, Bin Width and MCA-Channels
[&#x2B06;](#XMAGIX)

In [ ]:
idict = {"manual": True, "manual_name":"Set"}
pt = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("parset"), value=widgets.Dropdown(options=currentPeakingTimes.round(2).tolist(), value=2.4, description="Peaking Time"))
bw = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("mca_bin_width"), value=widgets.FloatText(value=1, description="Bin Width"))
nMCA = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("number_mca_channels"), value=widgets.FloatText(value=8192, description="No. MCA"))
ui1 = widgets.HBox([pt, bw, nMCA])
ui1

## 3.4. Setting Thresholds
[&#x2B06;](#XMAGIX)

* Trigger: 0-4095
* Baseline: 0-4095
* Energy: 0-4095

In [ ]:
th_trigger = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("trigger_threshold"), value=widgets.FloatText(value=150, description="Trigger Thr."))
th_baseline = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("baseline_threshold"), value=widgets.FloatText(value=120, description="Baseline Thr."))
th_energy = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("energy_threshold"), value=widgets.FloatText(value=0, description="Energy Thr."))
ui2 = widgets.HBox([th_trigger, th_baseline, th_energy])
ui2

## 3.5. Setting Gain
[&#x2B06;](#XMAGIX)

$Base Gain = \frac{1184}{DynRange[keV] \cdot Preamp Gain [\frac{mV}{keV}]}$

* Gain: 1-100 (4.4994)
* Gain Trim: 0.5-2.0 (-1)
* Gap Time: (0.3)

In [ ]:
gain = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("gain"), value=widgets.FloatText(value=4.4994, description="Gain"))
gain_trim = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("gain_trim"), value=widgets.FloatText(value=1, description="Gain Trim"))
gap_time = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("trigger_gap_time"), value=widgets.FloatText(value=.3, description="Gap Time"))
ui3 = widgets.HBox([gain, gain_trim, gap_time])
ui3

# 4. Starting and Stopping a Run
[&#x2B06;](#XMAGIX)

In [ ]:
#out = widgets.Output(layout={'border': '1px solid black'})
frt = widgets.interactive(xmagix.fixedRealtimeRun, {"manual": True, "manual_name": "Run"}, realtime=widgets.IntText(value=30, description="Time"), clearMca=widgets.Checkbox(value=True, name="Clear MCA"))
foc = widgets.interactive(xmagix.fixedEventsRun, {"manual": True, "manual_name": "Run", "enabled": False}, counts=widgets.IntText(value=10000, description="Counts"), clearMca=widgets.Checkbox(value=True, name="Clear MCA"))
sr = widgets.interactive(xmagix.stopRun, {"manual": True, "manual_name": "Stop Run"}, stopmessage=widgets.fixed("Stopped"))
ui4 = widgets.HBox([frt, foc, sr])
ui4

In [ ]:
xmagix.stopRun()

# 5. Pull Data and Plot
[&#x2B06;](#XMAGIX)

In [ ]:
cmca = xmagix.pullMcaData()
mca = np.ctypeslib.as_array(cmca)

In [ ]:
# Nightowl mode
plt.style.use("default")
plt.style.use("dark_background")
cm = 1/2.54 # inch to cm, metric like brrr
size = 60

titles = [f"nMCA {len(mca)}", "Bins", "Counts"]
fig, ax = plt.subplots(1, 1, figsize=(40*cm, 25*(9/16)*cm))
ax.set(
    # xlim=(-0.01*len(mca), 1.01*len(mca)), ylim=(-0.01*np.max(mca), 1.01 * np.max(mca)),
    xlabel=titles[1], ylabel=titles[2])
    # xticks=[0, len(mca)], yticks=[0, np.max(mca)],)
ax.set_yscale("log")
x = np.arange(0, len(mca), 1)
y = np.vstack([mca])
ax.stackplot(x, y, labels=["Empty"])
ax.legend()
plt.tight_layout()

if not os.path.exists("./tmp/"):
    os.makedirs("./tmp/") 
plt.savefig("./tmp/plot.svg")
plt.savefig("./tmp/plot.png")
SVG("./tmp/plot.svg")

In [ ]:
# Nightowl mode
plt.style.use("default")
plt.style.use("dark_background")
cm = 1/2.54 # inch to cm, metric like brrr
size = 60

titles = [f"nMCA {len(mca)}", "Bins", "Counts"]
fig, ax = plt.subplots(1, 1, figsize=(40*cm, 25*(9/16)*cm))
ax.set(
    # xlim=(-0.01*len(mca), 1.01*len(mca)), ylim=(-0.01*np.max(mca), 1.01 * np.max(mca)),
    xlabel=titles[1], ylabel=titles[2],
    # xticks=[0, len(mca)], yticks=[0, np.max(mca)],
    title=titles[0])
x = np.arange(0, len(mca), 1)
ax.set_yscale("log")
ax.plot(x, mca, linewidth=.5)
plt.tight_layout()

if not os.path.exists("./tmp/"):
    os.makedirs("./tmp/") 
plt.savefig("./tmp/plot.svg")
plt.savefig("./tmp/plot.png")
SVG("./tmp/plot.svg")

In [ ]:
csvPath = input()
# csvPath = "Pb_XRF"
if not os.path.exists(f"./{csvPath}"):
    os.makedirs(f"./{csvPath}")
os.rename("./tmp/plot.svg", f"./{csvPath}/{csvPath}.svg")
os.rename("./tmp/plot.png", f"./{csvPath}/{csvPath}.png")
with open(f"./{csvPath}/{csvPath}.csv", "x") as f:
    values.to_csv(f, sep=";", index=False)
    mcapd = pd.DataFrame(mca)
    f.write("BEGIN_OF_DATA")
    mcapd.to_csv(f, sep=";")
    f.write("END_OF_DATA")

In [ ]:
plt.style.use("default")
plt.style.use("dark_background")
fig, ax = plt.subplots(1,1)

print(help(ax))